In [11]:
v = GF(13)

In [12]:
P.<x> = PolynomialRing(v)

In [13]:
P

Univariate Polynomial Ring in x over Finite Field of size 13

In [16]:
v.primitive_element().order()

13

In [19]:
for elem in v:
    print(f"elem: {elem}")
    print(f"powers: {set(elem^i for i in range(1, 13))}")

elem: 0
powers: {0}
elem: 1
powers: {1}
elem: 2
powers: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
elem: 3
powers: {9, 3, 1}
elem: 4
powers: {1, 3, 4, 9, 10, 12}
elem: 5
powers: {8, 1, 12, 5}
elem: 6
powers: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
elem: 7
powers: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
elem: 8
powers: {8, 1, 12, 5}
elem: 9
powers: {9, 3, 1}
elem: 10
powers: {1, 3, 4, 9, 10, 12}
elem: 11
powers: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
elem: 12
powers: {1, 12}


In [20]:
delta = 7
b = 3

In [24]:
primitive_element = 2
gen_poly = prod((x - primitive_element ** i) for i in range(b, b + delta - 1))
gen_poly

x^6 + 3*x^5 + 11*x^4 + 9*x^3 + 12*x^2 + 4*x + 5

In [26]:
# выход канала 
y = 6 * x^11 + x^10 + 2*x^8 + x^7 + 4*x^6 + 5*x^5 + 4*x^4 + 2*x^3 + 10*x^2 + 12*x + 7

In [27]:
tau = (delta - 1) / 2
tau

3

In [44]:
rs = []
rs.append(x ^ (2 * tau))

In [45]:
S = sum(y(primitive_element ** (b + i)) * x ^ i for i in range(0, delta - 1))
S

10*x^5 + 12*x^4 + 2*x^3 + 8*x^2 + x + 10

In [46]:
rs.append(S)
rs

[x^6, 10*x^5 + 12*x^4 + 2*x^3 + 8*x^2 + x + 10]

In [47]:
qs = []

In [48]:
q, new_r = rs[-2].quo_rem(rs[-1])
rs.append(new_r)
qs.append(q)
rs

In [49]:
q, new_r = rs[-2].quo_rem(rs[-1])
rs.append(new_r)
qs.append(q)
rs

[x^6,
 10*x^5 + 12*x^4 + 2*x^3 + 8*x^2 + x + 10,
 8*x^4 + x^3 + 11*x^2 + 9*x + 9,
 6*x^3 + 10*x^2 + 4]

In [50]:
q, new_r = rs[-2].quo_rem(rs[-1])
rs.append(new_r)
qs.append(q)
rs

[x^6,
 10*x^5 + 12*x^4 + 2*x^3 + 8*x^2 + x + 10,
 8*x^4 + x^3 + 11*x^2 + 9*x + 9,
 6*x^3 + 10*x^2 + 4,
 7*x^2 + 8*x + 10]

In [51]:
qs

[4*x + 3, 11*x + 5, 10*x + 3]

In [53]:
U = prod(matrix([[-q, 1], [1, 0]]) for q in qs)
U

[2*x^3 + x^2 + 2*x + 1         5*x^2 + x + 3]
[      6*x^2 + 5*x + 3               2*x + 8]

In [55]:
L = U[0, 0]
L

2*x^3 + x^2 + 2*x + 1

In [57]:
L.roots()

[(8, 1), (6, 1), (5, 1)]

In [78]:
for (root, _) in L.roots():
    t = mod(root^(-1), 13)
    print(f"Error at position {discrete_log(v.gen() * t, v.gen() * 2)}")

Error at position 9
Error at position 7
Error at position 3


In [79]:
L.derivative()

6*x^2 + 2*x + 2

In [81]:
Gamma = rs[-1]
Gamma

7*x^2 + 8*x + 10

In [ ]:
Дальше мы делаем Форни

In [84]:
for (root, _) in L.roots():
    X = root^(-1)
    print(f"E_i: {-X^(1 - b) * Gamma(X^(-1)) / L.derivative()(X^(-1))}")

E_i: 11
E_i: 8
E_i: 5


In [ ]:
# Берлекэмп-Месси

In [136]:
Lambda = x^0 * v.gen()
r = 1
m = 0
L = 0
B = x^0 * v.gen()

In [137]:
while r <= delta - 1:
    Delta_r = 0
    for j in range(0, min(L+1, Lambda.degree()+1)):
        Delta_r += Lambda.list()[j] * S.list()[r - 1 - j]
    if Delta_r == 0:
        r += 1
        continue
    T = Lambda - Delta_r * x^(r-m) * B
    if 2 * L > r - 1:
        Lambda = T
    else:
        B = Lambda / Delta_r
        Lambda = T
        L = r - L
        m = r
    r += 1

In [139]:
Lambda

2*x^3 + x^2 + 2*x + 1

In [140]:
S.coefficients?

Docstring:     
   Return the coefficients of the monomials appearing in self. If
   "sparse=True" (the default), it returns only the non-zero
   coefficients. Otherwise, it returns the same value as
   "self.list()". (In this case, it may be slightly faster to invoke
   "self.list()" directly.)

   EXAMPLES:

      sage: _.<x> = PolynomialRing(ZZ)
      sage: f = x^4+2*x^2+1
      sage: f.coefficients()
      [1, 2, 1]
      sage: f.coefficients(sparse=False)
      [1, 0, 2, 0, 1]
Init docstring: Initialize self.  See help(type(self)) for accurate signature.
File:           /usr/lib/python3.10/site-packages/sage/rings/polynomial/polynomial_element.pyx
Type:           builtin_function_or_method
